https://huggingface.co/docs/transformers/tasks/translation

In [2]:
!pip install transformers datasets evaluate sacrebleu

     -------------------------------------- 468.7/468.7 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 81.4/81.4 kB 2.3 MB/s eta 0:00:00
     -------------------------------------- 118.9/118.9 kB 6.8 MB/s eta 0:00:00
     -------------------------------------- 134.3/134.3 kB 3.9 MB/s eta 0:00:00
     -------------------------------------- 199.8/199.8 kB 3.1 MB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 145.4/145.4 kB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 20.6/20.6 MB 4.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.8.1
    Uninstalling huggingface-hub-0.8.1:
      Successfully uninstalled huggingface-hub-0.8.1



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to C:\Users\andru/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [1]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

c:\Users\andru\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [2]:
source_lang = "en"
target_lang = "tok"
prefix = "translate English to Toki-pona: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [3]:
import json

In [13]:
with open("C:/Users/andru\Documents\GitHub/toki-pona-translator\corpora\eng_tok.json", 'r', encoding="utf-8") as f:
    sentences = json.loads(f.read())
    sentences = list(sentences["train"])

tokenized_sentences = map(preprocess_function, sentences)

TypeError: string indices must be integers

In [10]:
tokenized_sentences